In [ ]:
import geopandas as gpd
import pandas as pd

# Caminho do shapefile
caminho = "data/PRODES_AMZ_Municipal_2023.shp"

# Ler o shapefile
gdf = gpd.read_file(caminho)

print("Colunas encontradas no shapefile:")
print(gdf.columns)


# Agrupar por estado e ano, somando área total desmatada
# Salvar como CSV

gdf.to_csv('data/desmatamento_por_estado.csv', index=False)

print("✅ Arquivo CSV agregado salvo com sucesso em data/desmatamento_por_estado.csv")



In [ ]:
import pandas as pd

url = "https://apisidra.ibge.gov.br/values/t/5938/n6/all/v/37/p/2021"
df = pd.read_json(url)
df.to_csv("data/pib_municipal_2023.csv", index=False)

print("✅ PIB municipal 2021 salvo com sucesso!")


In [ ]:
import pandas as pd
import os

# Caminho da base original
caminho = "data/desmatamento_por_estado.csv"

# Criar pasta de saída
os.makedirs("data/limpos", exist_ok=True)

# 1️⃣ Ler a base
df = pd.read_csv(caminho)
print("✅ Base carregada com sucesso!")
print("📊 Linhas:", len(df))

# 2️⃣ Manter apenas colunas importantes
colunas_importantes = [
    "fid",
    "state",
    "main_class",
    "class_name",
    "image_date",
    "year",
    "area_km",
    "uuid"
]
df = df[colunas_importantes]

print("🔹 Colunas mantidas:", list(df.columns))

# 3️⃣ Corrigir tipos de dados
df["year"] = df["year"].astype(int)
df["area_km"] = df["area_km"].astype(float)
df["image_date"] = pd.to_datetime(df["image_date"], errors="coerce")

# 4️⃣ Filtrar apenas o ano de 2021
df = df.loc[df["year"] == 2021]

# 5️⃣ Remover linhas sem área ou ano válidos
df = df.dropna(subset=["area_km", "year"])

# 6️⃣ Padronizar nomes das colunas
df.columns = df.columns.str.lower()

# 7️⃣ Salvar versão limpa
output = "data/limpos/desmatamento_2021.csv"
df.to_csv(output, index=False, encoding="utf-8-sig")


In [ ]:
import pandas as pd
import re

# caminho do arquivo original
caminho = "data/pib_municipal_2021.csv"

# ler a base
df = pd.read_csv(caminho)

# renomear colunas para facilitar
df = df.rename(columns={
    "V": "pib_mil_reais",
    "D1N": "municipio_nome"
})

# extrair a sigla do estado do nome do município (depois do " - ")
df["UF"] = df["municipio_nome"].apply(lambda x: re.search(r"-\s*([A-Z]{2})$", x).group(1) if re.search(r"-\s*([A-Z]{2})$", x) else None)

# converter PIB para número
df["pib_mil_reais"] = pd.to_numeric(df["pib_mil_reais"], errors="coerce")

# agrupar por UF e somar
pib_estadual = df.groupby("UF", as_index=False)["pib_mil_reais"].sum()

# criar coluna em bilhões para facilitar a leitura
pib_estadual["pib_bilhoes"] = (pib_estadual["pib_mil_reais"] / 1_000_000).round(2)

# salvar resultado
pib_estadual.to_csv("data/pib_estadual_2021.csv", index=False, encoding="utf-8-sig")

print("✅ PIB estadual 2021 calculado com sucesso!")
print(pib_estadual)
